In [ ]:
# Importamos las librerías necesarias

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import pyeeg as pe
import pickle as pickle
import os
import time

# Definimos los canales que queremos analizar, en este caso son los 8 canales de un sistema OpenBCI
canales = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
# Las etiquetas en el comentario (Fp1, Fp2, C3, C4, etc.) indican las posiciones correspondientes de los electrodos EEG.

# Frecuencias de las bandas para obtener la Densidad Espectral de Potencia (PSD)
banda = [.5, 4, 8, 12, 30, 45]
# Estas representan las diferentes bandas de frecuencia para analizar los datos EEG:
# - Delta: de 0.5 Hz a 4 Hz
# - Theta: de 4 Hz a 8 Hz
# - Alpha: de 8 Hz a 12 Hz
# - Beta: de 12 Hz a 30 Hz
# - Gamma Baja: de 30 Hz a 45 Hz
# - Gamma Alta: por encima de 45 Hz

# División de tiempo para crear épocas de 2 segundos (downsampleado a 128 Hz)
tamano_ventana = 128 * 5 # Cada época tendrá una duración de 5 segundos (128 muestras por segundo * 5 segundos)

# Actualización del tiempo en ciertos pasos
tamaño_paso = 16 # La actualización se realiza cada 0.125 segundos (16 muestras por segundo)

# Tasa de muestreo de los datos EEG
tasa_muestreo = 128 # Los datos EEG se muestrean a 128 Hz (128 muestras por segundo)

# Para cargar todos los datos de los sujetos
lista_sujetos = ['01', '02', '03', ..., '32'] # Lista de sujetos (presumiblemente datos EEG de 32 sujetos diferentes)
# Reemplaza '...' con los códigos reales de los sujetos (por ejemplo, '01', '02', '03', etc.)

# Definimos una función para obtener las bandas EEG mediante el procesamiento de la Transformada Rápida de Fourier (FFT)
def Procesamiento_FFT(sub, canales, banda, tamano_ventana, tamaño_paso, tasa_muestreo):
    # Variable para almacenar los metadatos para cada época
    meta = []

    # Abrimos y leemos los datos EEG usando pickle
    with open('datos\s' + sub + '.dat', 'rb') as archivo:
        sujeto = pickle.load(archivo, encoding='latin1')

        for i in range(0, 40):
            datos = sujeto["data"][i] # Datos EEG para una prueba específica
            etiquetas = sujeto["labels"][i] # Etiquetas para cada paso de tiempo de la prueba
            inicio = 0

            # Iteramos sobre los datos de la prueba con los tamaños de ventana y paso especificados
            while inicio + tamano_ventana < datos.shape[1]:
                meta_array = []
                meta_data = [] # Vector meta para el análisis de potencia de banda

                # Iteramos sobre los canales EEG especificados para el análisis
                for j in canales:
                    X = datos[j][inicio : inicio + tamano_ventana] # Segmentamos los datos crudos durante 5 segundos
                    Y = pe.bin_power(X, banda, tasa_muestreo) # Calculamos la Densidad Espectral de Potencia (PSD) para la ventana actual
                    meta_data = meta_data + list(Y[0]) # Agregamos los valores de PSD al vector meta_data

                meta_array.append(np.array(meta_data)) # Convertimos meta_data en un array y lo agregamos a meta_array
                meta_array.append(etiquetas) # Agregamos la etiqueta correspondiente a meta_array

                meta.append(np.array(meta_array)) # Convertimos meta_array en un array y lo agregamos a la lista meta
                inicio = inicio + tamaño_paso # Movemos la ventana utilizando el tamaño de paso especificado

        # Convertimos la lista meta en un array NumPy para un procesamiento adicional
        meta = np.array(meta)

        # Guardamos los datos procesados del sujeto en un archivo usando la función de guardado de NumPy
        np.save('out\s' + sub, meta, allow_pickle=True, fix_imports=True)

# Listas para almacenar datos de los conjuntos de entrenamiento, prueba y validación
datos_entrenamiento = []
etiquetas_entrenamiento = []
datos_prueba = []
etiquetas_prueba = []
datos_validacion = []
etiquetas_validacion = []

# Cambiamos el comportamiento de carga de NumPy para allow_pickle=True
np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

# Iteramos sobre cada sujeto en la lista de sujetos
for sujeto in lista_sujetos:
    # Cargamos los datos procesados del sujeto
    with open(os.path.join('out', f's{sujeto}.npy'), 'rb') as archivo:
        sub = np.load(archivo, allow_pickle=True)

        # Iteramos sobre cada época en los datos del sujeto
        for i in range(sub.shape[0]):
            if i % 8 == 0:
                datos_prueba.append(sub[i][0])
                etiquetas_prueba.append(sub[i][1])
            elif i % 8 == 1:
                datos_validacion.append(sub[i][0])
                etiquetas_validacion.append(sub[i][1])
            else:
                datos_entrenamiento.append(sub[i][0])
                etiquetas_entrenamiento.append(sub[i][1])

# Restauramos el comportamiento de carga original de NumPy
np.load = np_load_old

# Guardamos los conjuntos de entrenamiento, prueba y validación en archivos separados
np.save(os.path.join('out', 'datos_entrenamiento.npy'), np.array(datos_entrenamiento), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'etiquetas_entrenamiento.npy'), np.array(etiquetas_entrenamiento), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'datos_prueba.npy'), np.array(datos_prueba), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'etiquetas_prueba.npy'), np.array(etiquetas_prueba), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'datos_validacion.npy'), np.array(datos_validacion), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'etiquetas_validacion.npy'), np.array(etiquetas_validacion), allow_pickle=True, fix_imports=True)

# Imprimimos las formas de los conjuntos de entrenamiento, prueba y validación resultantes
print("Conjunto de entrenamiento:", np.array(datos_entrenamiento).shape, np.array(etiquetas_entrenamiento).shape)
print("Conjunto de prueba:", np.array(datos_prueba).shape, np.array(etiquetas_prueba).shape)
print("Conjunto de validación:", np.array(datos_validacion).shape, np.array(etiquetas_validacion).shape)
